In [ ]:
import pygrib
import pandas as pd
import numpy as np
import os

# Eindhoven
eindhoven_lat = 51.4416
eindhoven_lon = 5.4697

# Folder grib files
grib_folder = r"C:\Users\20193362\Desktop\harm40_v1_p1_2024052706"

# List needed parameters, see code matrix KNMI
parameters = ['11', '33', '34', '117'] 

# Initialize list to hold the data
data_list = []

# Loop over each file 
for file_name in os.listdir(grib_folder):
    if file_name.endswith('_GB'):
        grib_file = os.path.join(grib_folder, file_name)
        grbs = pygrib.open(grib_file)
        
        # Retrieve the lat/lon grid
        first_message = grbs.message(1)
        lats, lons = first_message.latlons()
        
        # Find the closest grid point
        distance = np.sqrt((lats - eindhoven_lat)**2 + (lons - eindhoven_lon)**2)
        min_index = distance.argmin()
        nearest_point_lat = lats.flat[min_index]
        nearest_point_lon = lons.flat[min_index]
        
        # Initialize a dictionary to hold the data for this file
        data_dict = {
            'file_name': file_name,
            'latitude': nearest_point_lat,
            'longitude': nearest_point_lon
        }
        
        # Extract data for each parameter
        for param in parameters:
            try:
                grb_message = grbs.select(parameterName=param)[0] # First instance
                parameter_name = param
                eindhoven_value = grb_message.values.flat[min_index]
                data_dict[parameter_name] = eindhoven_value
            except (IndexError, ValueError):
                data_dict[parameter_name] = np.nan # When parameter is not found in grib file
        
        grbs.close()
        
        # Append dictionary to list
        data_list.append(data_dict)

# Convert list of dictionaries to DF
all_data = pd.DataFrame(data_list)

print(all_data)

In [13]:
import logging
import os
import sys

import requests

logging.basicConfig()
logger = logging.getLogger(__name__)
logger.setLevel(os.environ.get("LOG_LEVEL", logging.INFO))


class OpenDataAPI:
    def __init__(self, api_token: str):
        self.base_url = "https://api.dataplatform.knmi.nl/open-data/v1"
        self.headers = {"Authorization": api_token}

    def __get_data(self, url, params=None):
        return requests.get(url, headers=self.headers, params=params).json()

    def list_files(self, dataset_name: str, dataset_version: str, params: dict):
        return self.__get_data(
            f"{self.base_url}/datasets/{dataset_name}/versions/{dataset_version}/files",
            params=params,
        )

    def get_file_url(self, dataset_name: str, dataset_version: str, file_name: str):
        return self.__get_data(
            f"{self.base_url}/datasets/{dataset_name}/versions/{dataset_version}/files/{file_name}/url"
        )


def download_file_from_temporary_download_url(download_url, filename):
    try:
        with requests.get(download_url, stream=True) as r:
            r.raise_for_status()
            with open(filename, "wb") as f:
                for chunk in r.iter_content(chunk_size=8192):
                    f.write(chunk)
    except Exception:
        logger.exception("Unable to download file using download URL")
        sys.exit(1)

    logger.info(f"Successfully downloaded dataset file to {filename}")


def main():
    api_key = "eyJvcmciOiI1ZTU1NGUxOTI3NGE5NjAwMDEyYTNlYjEiLCJpZCI6ImE1OGI5NGZmMDY5NDRhZDNhZjFkMDBmNDBmNTQyNjBkIiwiaCI6Im11cm11cjEyOCJ9"
    dataset_name = "harmonie_arome_cy40_p1"
    dataset_version = "0.2"
    logger.info(f"Fetching latest file of {dataset_name} version {dataset_version}")

    api = OpenDataAPI(api_token=api_key)

    # sort the files in descending order and only retrieve the first file
    params = {"maxKeys": 1, "orderBy": "created", "sorting": "desc"}
    response = api.list_files(dataset_name, dataset_version, params)
    if "error" in response:
        logger.error(f"Unable to retrieve list of files: {response['error']}")
        sys.exit(1)

    latest_file = response["files"][0].get("filename")
    logger.info(f"Latest file is: {latest_file}")

    # fetch the download url and download the file
    response = api.get_file_url(dataset_name, dataset_version, latest_file)
    download_file_from_temporary_download_url(response["temporaryDownloadUrl"], latest_file)


if __name__ == "__main__":
    main()

INFO:__main__:Fetching latest file of harmonie_arome_cy40_p1 version 0.2
INFO:__main__:Latest file is: harm40_v1_p1_2024052706.tar
INFO:__main__:Successfully downloaded dataset file to harm40_v1_p1_2024052706.tar
